In [ ]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
# !tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.5.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Crear la sesión de Spark con configuraciones para evitar que se apague
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Regresion") \
    .config("spark.driver.idleTimeout", "3600s") \
    .config("spark.network.timeout", "3600s") \
    .config("spark.executor.memory", "6g") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .getOrCreate()

# Configuración adicional para mejorar la salida de las tablas
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

# Verificar la sesión de Spark
spark

25/02/12 20:40:30 WARN Utils: Your hostname, codespaces-8b4e98 resolves to a loopback address: 127.0.0.1; using 10.0.0.228 instead (on interface eth0)
25/02/12 20:40:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/12 20:40:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import re
from datetime import datetime

def process_chat(rdd):
    messages = []
    current_message = None
    current_text = []
    
    for line in rdd.collect():
        # Patrón para detectar nuevo mensaje
        pattern = r'(\d{1,2}/\d{1,2}/\d{2})\s(\d{1,2}:\d{1,2}\s[ap]\.\sm\.)\s-\s([^:]+):\s(.+)'
        match = re.match(pattern, line)
        
        if match:
            # Si hay un mensaje anterior, guardarlo
            if current_message:
                fecha, hora, miembro, _ = current_message
                mensaje_completo = ' '.join(current_text)
                messages.append((
                    datetime.strptime(fecha, '%d/%m/%y').strftime('%Y-%m-%d'),
                    hora.strip(),
                    miembro.strip(),
                    mensaje_completo.strip()
                ))
            
            # Iniciar nuevo mensaje
            current_message = match.groups()
            current_text = [match.group(4)]
        else:
            # Agregar línea al mensaje actual
            if current_message:
                current_text.append(line.strip())
    
    # Procesar el último mensaje
    if current_message:
        fecha, hora, miembro, _ = current_message
        mensaje_completo = '\n'.join(current_text)
        messages.append((
            datetime.strptime(fecha, '%d/%m/%y').strftime('%Y-%m-%d'),
            hora.strip(),
            miembro.strip(),
            mensaje_completo.strip()
        ))
    
    return messages

# Crear DataFrame
rdd = spark.sparkContext.textFile("chat.txt")
messages = process_chat(rdd)

schema = StructType([
    StructField("Fecha", StringType(), True),
    StructField("Hora", StringType(), True),
    StructField("Miembro", StringType(), True),
    StructField("Mensaje", StringType(), True)
])

df = spark.createDataFrame(messages, schema)

# Mostrar resultado
df.show(500)

+----------+-----------+---------+--------------------+
|     Fecha|       Hora|  Miembro|             Mensaje|
+----------+-----------+---------+--------------------+
|2024-08-31| 1:17 a. m.|  Maryori|Kevin me contó qu...|
|2024-08-31| 1:17 a. m.|Maricielo|            Ay amiga|
|2024-08-31| 1:18 a. m.|  Maryori|Que cólera amiga ...|
|2024-08-31| 1:18 a. m.|Maricielo|A estrella le eje...|
|2024-08-31| 1:18 a. m.|Maricielo|               Bueno|
|2024-08-31| 1:18 a. m.|Maricielo|       Ni modo amiga|
|2024-08-31| 1:19 a. m.|Maricielo|Nose si llorar o ...|
|2024-08-31| 1:19 a. m.|Maricielo|Pero bueno a segu...|
|2024-08-31| 1:21 a. m.|  Maryori|Imagínate amiga e...|
|2024-08-31| 1:22 a. m.|Maricielo|Y sabes amiga a l...|
|2024-08-31| 1:22 a. m.|Maricielo|    Y nosotros móvil|
|2024-08-31| 1:22 a. m.|Maricielo|Si fue asi yo tmb...|
|2024-08-31| 1:23 a. m.|Maricielo|Y me da cólera po...|
|2024-08-31| 1:23 a. m.|Maricielo|Y en cambio tu y ...|
|2024-08-31| 1:24 a. m.|Maricielo|Por eso amiga 

In [12]:
from pyspark.sql.functions import size, split, when, length

# Agregar columnas de análisis de mensajes
df_analizado = df.withColumn(
    "Cantidad_Palabras", 
    size(split("Mensaje", " "))
).withColumn(
    "Respuesta_Corta",
    when(
        (length("Mensaje") <= 20) & 
        (size(split("Mensaje", " ")) <= 3), 
        1
    ).otherwise(0)
)

# Mostrar el resultado
df_analizado.select("Fecha", "Hora", "Miembro", "Mensaje", "Cantidad_Palabras", "Respuesta_Corta").show(5, truncate=False)


+----------+----------+---------+--------------------------------------------------------------------------+-----------------+---------------+
|Fecha     |Hora      |Miembro  |Mensaje                                                                   |Cantidad_Palabras|Respuesta_Corta|
+----------+----------+---------+--------------------------------------------------------------------------+-----------------+---------------+
|2024-08-31|1:17 a. m.|Maryori  |Kevin me contó que el profe se puso especial justo con ustedes 😢         |13               |0              |
|2024-08-31|1:17 a. m.|Maricielo|Ay amiga                                                                  |2                |1              |
|2024-08-31|1:18 a. m.|Maryori  |Que cólera amiga dicen que siempre es así cuando revisa                   |10               |0              |
|2024-08-31|1:18 a. m.|Maricielo|A estrella le ejecuto y dijo ya aprobada yo ejecute y me pidió 84000 cosas|15               |0              |


In [14]:

# Celda 1: Análisis de expresiones negativas
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType

# Función para detectar sentimiento (simplificada - deberías usar un modelo más robusto)
def analizar_sentimiento(texto):
    palabras_negativas = ['mal', 'terrible', 'horrible', 'peor', 'no', 'nunca', 'problema']
    palabras_positivas = ['bien', 'excelente', 'genial', 'bueno', 'gracias', 'si', 'claro']
    
    texto = texto.lower()
    if any(palabra in texto for palabra in palabras_negativas):
        return 'Negativo'
    elif any(palabra in texto for palabra in palabras_positivas):
        return 'Positivo'
    return 'Neutral'

# Registrar UDFs
sentimiento_udf = udf(analizar_sentimiento, StringType())
expresiones_negativas_udf = udf(lambda texto: 1 if analizar_sentimiento(texto) == 'Negativo' else 0, IntegerType())

# Crear DataFrame con expresiones negativas
df_negativo = df_analizado.withColumn("Sentimiento", sentimiento_udf("Mensaje")) \
                         .withColumn("Expresiones_negativas", expresiones_negativas_udf("Mensaje"))

In [15]:
# Celda 2: Análisis de expresiones positivas
expresiones_positivas_udf = udf(lambda texto: 1 if analizar_sentimiento(texto) == 'Positivo' else 0, IntegerType())

# Crear DataFrame con expresiones positivas
df_positivo = df_analizado.withColumn("Sentimiento", sentimiento_udf("Mensaje")) \
                         .withColumn("Expresiones_positivas", expresiones_positivas_udf("Mensaje"))

In [16]:
# Celda 3: Análisis de temas académicos
def detectar_tema_academico(texto):
    palabras_academicas = ['tarea', 'examen', 'clase', 'profesor', 'curso', 'estudiar', 
                          'práctica', 'parcial', 'final', 'investigación', 'informe']
    texto = texto.lower()
    return 1 if any(palabra in texto for palabra in palabras_academicas) else 0

temas_academicos_udf = udf(detectar_tema_academico, IntegerType())

# Crear DataFrame con temas académicos
df_academico = df_analizado.withColumn("Temas_academicos", temas_academicos_udf("Mensaje"))

# Mostrar resultados
print("DataFrame con expresiones negativas:")
df_negativo.show(5, truncate=False)

print("\nDataFrame con expresiones positivas:")
df_positivo.show(5, truncate=False)

print("\nDataFrame con temas académicos:")
df_academico.show(5, truncate=False)

DataFrame con expresiones negativas:


+----------+----------+---------+--------------------------------------------------------------------------+-----------------+---------------+-----------+---------------------+
|Fecha     |Hora      |Miembro  |Mensaje                                                                   |Cantidad_Palabras|Respuesta_Corta|Sentimiento|Expresiones_negativas|
+----------+----------+---------+--------------------------------------------------------------------------+-----------------+---------------+-----------+---------------------+
|2024-08-31|1:17 a. m.|Maryori  |Kevin me contó que el profe se puso especial justo con ustedes 😢         |13               |0              |Neutral    |0                    |
|2024-08-31|1:17 a. m.|Maricielo|Ay amiga                                                                  |2                |1              |Neutral    |0                    |
|2024-08-31|1:18 a. m.|Maryori  |Que cólera amiga dicen que siempre es así cuando revisa                   |10      

In [18]:
# Calcular porcentajes
def calcular_porcentajes(df, columna):
    total = df.count()
    positivos = df.filter(df[columna] == 1).count()
    return (positivos / total) * 100

print(f"Porcentaje de expresiones negativas: {calcular_porcentajes(df_negativo, 'Expresiones_negativas')}%")
print(f"Porcentaje de expresiones positivas: {calcular_porcentajes(df_positivo, 'Expresiones_positivas')}%")
print(f"Porcentaje de temas académicos: {calcular_porcentajes(df_academico, 'Temas_academicos')}%")

df.show(5, truncate=False)

Porcentaje de expresiones negativas: 18.681318681318682%
Porcentaje de expresiones positivas: 17.36263736263736%
Porcentaje de temas académicos: 6.373626373626373%
+----------+----------+---------+--------------------------------------------------------------------------+
|Fecha     |Hora      |Miembro  |Mensaje                                                                   |
+----------+----------+---------+--------------------------------------------------------------------------+
|2024-08-31|1:17 a. m.|Maryori  |Kevin me contó que el profe se puso especial justo con ustedes 😢         |
|2024-08-31|1:17 a. m.|Maricielo|Ay amiga                                                                  |
|2024-08-31|1:18 a. m.|Maryori  |Que cólera amiga dicen que siempre es así cuando revisa                   |
|2024-08-31|1:18 a. m.|Maricielo|A estrella le ejecuto y dijo ya aprobada yo ejecute y me pidió 84000 cosas|
|2024-08-31|1:18 a. m.|Maricielo|Bueno                                    